<a href="https://colab.research.google.com/github/kerryqliu/stats170/blob/irene/document-ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import json

from google.cloud import storage
from google.oauth2 import service_account

In [7]:
project_id = 'sapient-stacker-308223'

In [8]:
import getpass

APIKEY = getpass.getpass()

··········


In [4]:
!pip install --upgrade pip

     |████████████████████████████████| 1.5MB 5.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [5]:
!pip install --upgrade google-api-python-client

     |████████████████████████████████| 6.6 MB 5.2 MB/s 
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.0.4
    Uninstalling google-auth-httplib2-0.0.4:
      Successfully uninstalled google-auth-httplib2-0.0.4
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.258 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 2.1.0 which is incompatible.


In [1]:
!pip install google-cloud-documentai

In [24]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/content/drive/Shareddrives/CAPSTONE/Resources/sapient-stacker-308223-836172bc8681.json"

In [25]:
!gcloud auth application-default print-access-token

ya29.c.Kp8B-ge1yv9jK9jNvSJw-XcWPpH9N4uz4__5QF2GPVKbjANWfXBjUG2gLXQurb4gx8IUwUjuBTA1lxNOUa_uYtuEAgYRjlfMCbUkvhaHVR81cYkgmZUKQz9pKUdghYiDuW0ppco_FqWRgxEErgrsqi1TiOw2vzBdNPAyVBjikV8_UOYaK9hUil8Mqv7FqKhI99j-plT77Sv_SARYnFLeAaXW


In [26]:
from google.cloud import documentai_v1beta2 as documentai

In [27]:
def main(
    project_id="sapient-stacker-308223",
    input_uri="gs://medical-documents/nerve 3.pdf",
):
    """Process a single document with the Document AI API, including
    text extraction and entity extraction."""

    client = documentai.DocumentUnderstandingServiceClient()

    gcs_source = documentai.types.GcsSource(uri=input_uri)

    # mime_type can be application/pdf, image/tiff,
    # and image/gif, or application/json
    input_config = documentai.types.InputConfig(
        gcs_source=gcs_source, mime_type="application/pdf"
    )

    # Location can be 'us' or 'eu'
    parent = "projects/{}/locations/us".format(project_id)
    request = documentai.types.ProcessDocumentRequest(
        parent=parent, input_config=input_config
    )

    document = client.process_document(request=request)

    # All text extracted from the document
    print("Document Text: {}".format(document.text))

    def _get_text(el):
        """Convert text offset indexes into text snippets."""
        response = ""
        # If a text segment spans several lines, it will
        # be stored in different text segments.
        for segment in el.text_anchor.text_segments:
            start_index = segment.start_index
            end_index = segment.end_index
            response += document.text[start_index:end_index]
        return response

    for entity in document.entities:
        print("Entity type: {}".format(entity.type_))
        print("Text: {}".format(_get_text(entity)))
        print("Mention text: {}\n".format(entity.mention_text))

In [29]:
main(input_uri="gs://medical-documents/nerve 2.pdf")

Document Text: Examination of the neck showed full range of movement with no bruits, masses,
thyromegaly or adenopathy. Auscultation of the heart and lungs was normal.
There was no organomegaly or adenopathy.
Examination of the extremities showed normal tone throughout. Tendon reflexes
were all absent apart from trace ankle jerks, which could only be obtained with
reinforcement. All other tendons reflexes were absent. All muscles
demonstrated normal strength apart from reduced 4+/5 right ankle dorsiflexion
strength and reduced 3/5 strength of the right toe extensors and the right
extensor hallucis longus. The right extensor digitorum brevis muscles were felt

Entity type: 
Text: demonstrated normal strength apart from reduced 4+/5 right ankle dorsiflexion
Mention text: reduced 4+/5 right ankle dorsiflexion

